In [35]:
# Imports
import requests
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Helper functions
#def cache_file(url):
#    """Download url and cache file locally for future use"""
#    fname = url.split('/')[-1]
#    if os.path.isfile(fname) is False:
#        req = requests.get(url)
#        # Write raw content in binary format
#       with open(fname, 'wb') as f:
#            f.write(req.content)
#    return fname


In [37]:
# Download data and construct a DataFrame
#url = 'https://iot.fvh.fi/opendata/uiras/uiras-meta.json'
#fname = cache_file(url)  # Delete file from local directory, if you want to use fresh data
#cols = ['readable_time', 'batt', 'lat', 'lon', 'rssi', 'temp_in', 
#        'temp_out1', 'temp_out2', 'temprh_rh', 'temprh_temp', 'dev-id']
#df = pd.read_csv(fname, compression='gzip', parse_dates=['readable_time'], usecols=cols, index_col='readable_time')
#df.index = df.index.rename('time') 
#df.tail()

In [38]:
#fname = "uiras-meta.json"


#import geopandas as gpd


#data = pd.read_json(fname)
#print(data.head())

In [166]:
from shapely.geometry import Point
from pyproj import CRS

# Reading the data into a dataframe
data = pd.read_json('https://iot.fvh.fi/opendata/uiras/uiras-meta.json', 
                    orient="index") 

# Creating shapely Point objects from the coordinates
data['geometry'] = data.apply(lambda row: 
                              Point(row['lon'], row['lat']), axis=1)

# Creating a geodataframe
geo = gpd.GeoDataFrame(data, 
                       geometry='geometry')

# Setting the CRS to EPSG:3857
geo = geo.set_crs(epsg=4326)
geo = geo.to_crs(epsg=3857)


In [168]:
# Reading the temperature data
temps = pd.read_csv("https://iot.fvh.fi/opendata/uiras/uiras-all-data.csv")
temps

,time,readable_time,analog1,analog2,batt,lat,lon,rssi,temp_in,temp_out1,temp_out2,temprh_rh,temprh_temp,dev-id
0,1580508499096,2020-01-31T22:08:19.096000Z,0.012,NaN,5.137,NaN,NaN,-105,2.39,1.62,NaN,NaN,NaN,70B3D57050001AB9
1,1580509100076,2020-01-31T22:18:20.076000Z,0.012,NaN,5.137,NaN,NaN,-110,2.47,1.62,NaN,NaN,NaN,70B3D57050001AB9
2,1580509702252,2020-01-31T22:28:22.252000Z,0.012,NaN,5.137,NaN,NaN,-111,2.55,1.56,NaN,NaN,NaN,70B3D57050001AB9
3,1580510302101,2020-01-31T22:38:22.101000Z,0.012,NaN,5.137,NaN,NaN,-108,2.47,1.62,NaN,NaN,NaN,70B3D57050001AB9
4,1580510903015,2020-01-31T22:48:23.015000Z,0.012,NaN,5.137,NaN,NaN,-110,2.36,1.56,NaN,NaN,NaN,70B3D57050001AB9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137234,1606492383605,2020-11-27T15:53:03.605000Z,NaN,NaN,5.231,NaN,NaN,-80,3.60,4.87,NaN,NaN,NaN,70B3D57050001BBE
137235,1606492397892,2020-11-27T15:53:17.892000Z,0.012,NaN,5.137,NaN,NaN,-110,2.01,3.56,NaN,NaN,NaN,70B3D57050001AB9
137236,1606492401184,2020-11-27T15:53:21.184000Z,NaN,NaN,4.682,NaN,NaN,-101,3.60,6.56,NaN,NaN,NaN,70B3D57050004C07
137237,1606492487220,2020-11-27T15:54:47.220000Z,NaN,NaN,4.660,NaN,NaN,-113,5.06,5.87,NaN,NaN,NaN,70B3D57050004FE6


In [163]:
########
#import contextily as ctx
#fig, ax = plt.subplots()

#geo.plot(ax=ax, legend=True)

# Add basemap
#ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=15)


In [162]:
##########
#ax = geo.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
#ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=10)


In [160]:
import folium

point_gjson = folium.features.GeoJson(geo, name="points")

m = folium.Map(location=[60.25, 24.8], zoom_start=10, control_scale=True)

point_gjson.add_to(m)

m


In [161]:
geo

,name,lat,lon,servicemap_url,site_url,fieldmap,geometry
70B3D57050001AB9,Pikkukosken uimaranta,60.227704,24.983821,https://palvelukartta.hel.fi/unit/41960,,NaN,POINT (2781186.232 8450609.098)
70B3D57050001BBE,Rastilan uimaranta,60.207977,25.114849,https://palvelukartta.hel.fi/fi/unit/40157,,NaN,POINT (2795772.202 8446187.952)
70B3D57050004D86,Pihlajasaari,60.140588,24.915700,https://palvelukartta.hel.fi/fi/unit/45606,,NaN,POINT (2773603.059 8431104.992)
70B3D57050004FB9,Hietaniemi (Ourit),60.171500,24.898300,https://palvelukartta.hel.fi/fi/unit/41717,http://www.tuk.fi,NaN,POINT (2771666.078 8438019.854)
70B3D57050004C07,Sompasauna,60.175742,24.975318,https://palvelukartta.hel.fi/fi/unit/54929,https://www.sompasauna.fi,NaN,POINT (2780239.682 8438969.276)
70B3D57050004DF8,Vasikkasaari,60.152320,25.015860,https://palvelukartta.hel.fi/fi/unit/50903,https://www.vasikkasaari.org,NaN,POINT (2784752.797 8433728.617)
70B3D57050004FE1,Herttoniemi (Tuorinniemen uimalaituri),60.185540,25.040120,https://palvelukartta.hel.fi/fi/unit/41791,,NaN,POINT (2787453.408 8441162.681)
70B3D57050004FE6,Vartiosaari (Reposalmen laituri),60.180100,25.068600,https://palvelukartta.hel.fi/fi/unit/57156,https://www.vartiosaari.fi,NaN,POINT (2790623.787 8439944.788)
70B3D57050004E0E,Marjaniemen uimaranta,60.198449,25.076416,https://palvelukartta.hel.fi/fi/unit/40386,,NaN,POINT (2791493.860 8444053.522)
70B3D5705000504F,Hanikan uimaranta (Espoo),60.127797,24.691871,https://palvelukartta.hel.fi/fi/unit/39583,,NaN,POINT (2748686.506 8428245.610)


In [164]:
######
#url = 'https://a.basemaps.cartocdn.com/light_all/10/24.6/60.8.png'
#import os
#import urllib

# Control figure size in here
#fig, ax = plt.subplots(figsize=(12,8))

# The formatting should follow: 'https://{s}.basemaps.cartocdn.com/{style}/{z}/{x}/{y}{scale}.png'
# Specify the style to use
#style = "rastertiles/voyager"
#cartodb_url = 'https://a.basemaps.cartocdn.com/%s/{z}/{x}/{y}.png' % style

# Plot the data from subset
#geo.plot(ax=ax, legend=True)

#credits = "testi"

# Add basemap with `OSM_A` style using zoom level of 14 
#ctx.add_basemap(ax, zoom=10, attribution=credits, source=cartodb_url)

# Crop the figure
#ax.set_xlim(2770000, 2785000)
#ax.set_ylim(8435000, 8442500)

